# Usage Instructions - MTTR Predictor
The MTTR (Mean Time to Resolution) predictor is an AI/ML based solution which predicts the time taken by a service agent to solve a specific ticket or incident request. The solution learns the efficiency, experience and workload management metrics for various ticket types solved by service agents to arrive at the predictions. The solution helps business in optimal ticket allocation leading to a low MTTR, shorter wait time, fewer open incidents leading to improved efficiency and SLA (Service Level Agreement) adherence.







## Contents

1. [Prequisites](#Prerequisite)
1. [Data Dictionary](#Data-Dictionary)
1. [Set Up The Environment](#Set-up-the-environment)
1. [Create The Model](#Create-Model)
1. [Batch Transform Job](#Batch-Transform-Job)

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.


### Input format
#### Input1:
Name of the file: <b>”Assigned.csv”</b><br>
This file contains historical incidents that have been resolved. The solution uses the following incident specific inputs to derive specific productivity measures such as efficiency, experience and workload management across incident types for incident managers to make the predictions.<br><br>

</ul>
<li> Request ID: Unique identifier for the request- alphanumeric e.g. SRV101_254859</li>
<li> Request Submitted Date and Time: The data and time when the request was submitted (Preferred format: YYYY-MM-DD HH:MM:SS)</li>

<li> Request Priority: Priority of the request e.g. High, Medium, Low</li>
<li> Request Resolved Date and Time: The date and time when the request was closed (Only for closed requests, Preferred format: YYYY-MM-DD HH:MM:SS)</li>
<li> Request Category: Type of request e.g. "Authentication issue", "Server failure issue", "Access grant request"</li>
<li> Request Status: Status of the request e.g. Open/Closed</li>
<li> Request Resolved By: Service Provider ID/name who solved the incident (for closed incidents) or whom incident has been assigned for resolution (for open incidents).</li>
</ul><br>
NOTE:
</ul>
<li>For Assigned requests, all the above data fields are mandatory ("Request resolved by" should NOT be blank).</li>
<li>Provide a minimum of 10000 records (of assigned requests) for better results</li>
</ul>

#### Input2:
<br>
Name of the file:<b> “Unassigned.csv”</b>
<ul>
	
<li>This file contains those incidents which have not been assigned to any service provider.</li>
 <li> This File also requires above mentioned incident specific inputs except the following two inputs:</li>
<li>Request Resolved Date and Time  and Request Resolved By</li>
</ul>

#### Input3:<br>
Name of the file: <b>“Test.csv”<br></b>
Requires following data field:
<ul>

 
 <li>Request ID: Unique identifier for the request- alphanumeric e.g. SRV101_254859</li>
 <li>Request Assigned To: Service provider name/Id (to whom incident potentially can be assigned) for whom MTTR will be calculated.</li>
</ul>    
NOTE: The Request ID should be from the Input2.


## Set up the environment


### Update Boto Client and AWS SDK

We are launching new APIs in SageMaker to support this new functionality. The next cell sets it up for you to invoke the new APIs.

### Private Beta Setup

The private beta is limited to us-east-2 region. The client we are creating below will be hard-coded to talk to our us-east-2 endpoint only.



### Sample input data

We have provided tag in sample input file for referrence and better understanding of data.

In [63]:
import pandas as pd
import zipfile
#from zipfile import Zipfile
zf=zipfile.ZipFile("input3.zip")
Assigned = pd.read_csv(zf.open('Assigned.csv'),parse_dates=['Request Submitted Date and Time','Request Resolved Date and Time'])
Unassigned=pd.read_csv(zf.open('Unassigned.csv'),parse_dates=['Request Submitted Date and Time'])
Test=pd.read_csv(zf.open('Test.csv'))

In [64]:
Assigned.head(5)

,Request ID,Request Resolved By,Request Submitted Date and Time,Request Priority,Request Resolved Date and Time,Request Category,Request Status
0,INC000003867518,nbkhk0o,2016-04-01 04:04:00,3-Low,2016-04-02 10:16:00,FAILURE_SOFTWARE_BITLOCKER \ ERROR MESSAGE,Closed
1,INC000003867920,zkfbpjz,2016-04-01 04:52:00,3-Low,2016-04-01 04:52:00,REQUEST_SERVICE DESK_TICKET STATUS \ STATUS,Closed
2,INC000003867983,zkq3hr0,2016-04-01 04:23:00,3-Low,2016-04-01 04:23:00,REQUEST_SERVICE DESK_TICKET STATUS \ STATUS,Closed
3,INC000003867989,nbkyvee,2016-04-01 04:10:00,3-Low,2016-04-01 04:31:00,REQUEST_SOFTWARE_BITLOCKER \ LAPTOP RECOVERY,Closed
4,INC000003868015,nbkntia,2016-04-01 05:27:00,3-Low,2016-04-01 05:31:00,FAILURE_SOFTWARE_MS INTERNET EXPLORER \ PROXY/...,Closed


In [65]:
Unassigned.head(5)

,Request ID,Request Submitted Date and Time,Request Priority,Request Category,Request Status
0,INC000003875851,2016-04-01 14:32:00,3-Low,FAILURE_SECURITY_ENABLE,Open
1,INC000003875855,2016-04-01 14:13:00,3-Low,FAILURE_SOFTWARE_MAS - MOBILE ACCESS SRVCS \ C...,Open
2,INC000003875863,2016-04-01 14:27:00,3-Low,REQUEST_SOFTWARE_BITLOCKER \ LAPTOP RECOVERY,Open
3,INC000003875866,2016-04-01 14:54:00,3-Low,REQUEST_SOFTWARE_QUALITY CENTER \ INSTALL/CONF...,Open
4,INC000003875867,2016-04-01 14:46:00,3-Low,REQUEST_SOFTWARE_WINDOWS \ PASSWORD-RESET,Open


In [66]:
Test.head(5)

,Request ID,Request Assigned To
0,INC000003875931,nbk0yuq
1,INC000003875904,abcd
2,INC000003875900,zkakurn
3,INC000003875917,zk5ogd1


In [67]:
# S3 prefix
import boto3
import re

import os
import numpy as np
import pandas as pd


In [68]:
mm = boto3.client('sagemaker', region_name='us-east-2', endpoint_url="https://sagemaker.us-east-2.amazonaws.com")

### Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [69]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role

sess = sage.Session()
role = get_execution_role()

## Create Model

Now we use the Model Package to create a model

In [70]:
model_package_arn = 'your-arn-number'

In [71]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [72]:
model = ModelPackage(model_package_arn=model_package_arn,
                    role = role,
                    sagemaker_session = sagemaker_session)

## Batch Transform Job

Now let's use the model built to run a batch inference job and verify it works.



In [73]:
import json 
import uuid


transformer = model.transformer(1, 'ml.m5.large')
transformer.transform('s3://your-bucket-name/your-folder-name/input.zip', content_type='application/zip')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

................ * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 270-278-763
169.254.255.130 - - [16/Apr/2020 15:37:44] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [16/Apr/2020 15:37:44] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [16/Apr/2020 15:37:44] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [16/Apr/2020 15:37:44] "#033GET /execution-parameters HTTP/1.1#033" 404 -
Incident Number                                  INC000003875931
Incident Submitted Dt (GMT)                  2016-04-01 14:05:00
Incident Priority                                          3-Low
final cat                      FAILURE_WORKSTATION_ERROR / ALERT
ticket status                                               Open
id_new                                             

### Decoding the output bucket
 s3://your-output-bucket-name/folder-name-inside-output-bucket

In [74]:
s3_conn = boto3.client("s3")
# Refer to above cell to decode the output bucket
bucket_name= transformer.output_path.rsplit('/')[2] #"your-output-bucket-name"
bucketFolder = transformer.output_path.rsplit('/')[3] #"folder-name-inside-output-bucket"
with open('Output.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/input.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [75]:
df = pd.read_csv("Output.csv")
df.head(10)

,SID,Request ID,Request Assigned To,MTTR
0,0,INC000003875931,nbk0yuq,3.0 minutes
1,1,INC000003875904,abcd,value not found :Average Incident Number Time ...
2,2,INC000003875900,zkakurn,5.0 minutes
3,3,INC000003875917,zk5ogd1,66.57 hours


### Output Interpretation:
| Request ID | Request Assigned To | MTTR |
|-------------|---------------|----------------|
| 12426        |  nbk0yuq         | 5 hours 2 minutes  |

The solution will provide MTTR (see column “MTTR”) w.r.t service provider (see the column “Request Assigned To ”).





## Creating the Live Endpoint

In [76]:
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit

role = get_execution_role()

sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()

In [77]:
content_type='application/zip'
model_name='your-model-package-name'
real_time_inference_instance_type='ml.m5.large'

In [78]:
model_package_arn = 'your-arn-number'

In [79]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [80]:
#Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type=content_type)
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [81]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

In [82]:
f = open('./input.zip', mode='rb')
data=f.read()
prediction = predictor.predict(data)
print(prediction)

b'SID,Request ID,Request Assigned To ,MTTR\n0,INC000003875931,nbk0yuq,3.0 minutes\n1,INC000003875904,abcd,value not found :Average Incident Number Time  : 0.38 hours\n2,INC000003875900,zkakurn,4.0 minutes\n3,INC000003875917,zk5ogd1,66.57 hours\n'


### Output Interpretation:
| Request ID | Request Assigned To | MTTR |
|-------------|---------------|----------------|
| 12426        |  nbk0yuq         | 5 hours 2 minutes  |

The solution will provide MTTR (see column “MTTR”) w.r.t service provider (see the column “Request Assigned To ”).
